# Wikipedia game
#### Dhwani Contractor

## Main Function: findLinkRoute
#### Input: 
startlink: Link of the starting page

endlink: Link of the end page

#### Output:
result: Sequence of the page names

## Other Functions:
### getPageName
#### Input: 
url: url of the link
#### Output: 
pname: Pagename of the website

### joinLink
#### Input: 
pagename: Pagename of the link
#### Output: 
url: The Wikipedia URL

### findLinks
#### Input: 
link: Link of the website 
#### Output: 
set :set of the pagename

In [14]:
from bs4 import BeautifulSoup
import urllib.request
from queue import Queue

In [15]:
#splits the url annd takes the part comes after "/wiki/"
def getPageName(url):
    pname=url.split("/wiki/",1)[1]
    return pname

In [16]:
#Joins the pagename with the wikipedia url
def JoinLink(pagename): 
    return "https://en.wikipedia.org/wiki/"+pagename

In [25]:
#Find all the links available on the entire webpage and 
#puts it in a set so that if any link is there more than once, it won't be the part of set again

def findLinks(link):
    s=set()
    resp = urllib.request.urlopen(link)
    soup = BeautifulSoup(resp,"lxml", from_encoding=resp.info().get_param('charset'))
    for link in soup.find_all('a', href=True):
        l=link['href']
        if l.startswith('/wiki/') and l.find(":")==-1 and l.count('/') == 2:
            pagename=getPageName(l)
            s.add(pagename)
    return s

In [26]:
def findLinkRoute(startlink,endlink):
    #Find the pagename of the startlink and endlink
    startPage=getPageName(startlink)
    endPage=getPageName(endlink)
    
    #q is a Que of the pages we need to check
    q = Queue()
    #Parent is a directory which will keep track of the child(as key) and its parent(as value)
    parent = {}
    #Found is a flag which will be true when the endpage will match
    found = False
    #Making the first page's parent null
    parent[startPage] = None
    print("Finding path... Please wait")
    #Finds the link of all the pages on the start page
    first_child=findLinks(startlink)
    
    #Put all the links in the queue 
    for i in first_child:
        parent[i] = startPage
        q.put(i)
        if i == -1:
            found = True
                
    #Searches the end page till the que is empty
    while not q.empty() and not found:
        #take the elements of the queue one by one
        currentpage=q.get()
        currentlink=JoinLink(currentpage)
        #Find all the links on the current webpage
        nextchild=findLinks(currentlink)
        
        #Save the parent page of the current page
        for i in nextchild:
            if i not in parent:
                parent[i] = currentpage
                q.put(i)
                if i == endPage:
                    #we found the link
                    found = True
                    break
       
    
    print("The path is being found, creating a path now..")
    #build the path now
    result = []
    temp = endPage
    #Trackback till the first page comes 
    while temp != startPage:
        if(temp):
            result.insert(0, parent[temp])
            temp = parent[parent[temp]]
        else:
            break
    result.insert(0, startPage) if result[0] != startPage else None
    result.append(endPage)
    return result

In [27]:
#Main

#Insert the start and end link here
startlink="https://en.wikipedia.org/wiki/Web_Bot"
endlink="https://en.wikipedia.org/wiki/Tax_holiday"
path=findLinkRoute(startlink,endlink)

#Print the path of the pages
for page in path[:-1]:
    print(page, sep=' ', end=' -> ', flush=True)
print(path[-1])

Finding path... Please wait
The path is being found, creating a path now..
Web_Bot -> Taxation_in_Iran -> Tax_holiday


# Conclusion

- Here, the applied algorithm finds the "Shortest Path" to reach the end page. Hence, the path is being completed just in 3 steps rather than the mentioned 4 steps path.


- The time taken to find the result was around 20-25 min. This time can be improved by putting the depth limit. Means, the algorithm will match the end page name just till maximum depth. If the page is not found then the algorithm will start processing the other nodes. Drawback of such implementation will be if the end page is in the deeper node than the algorithm won't be able to find the solution. 